In [167]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

## 1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars

In [44]:
#Activating the chrome browser
driver = webdriver.Chrome("chromedriver.exe")

In [45]:
#Getting amazon page
driver.get('https://www.amazon.in/')
#Giving input
search_item= input('Enter the product that has to be searched : ')

Enter the product that has to be searched : Laptop


In [46]:
#Scraping search bar Xpath and clicking on search icon
search_bar = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')   
search_bar.clear()                                               
search_bar.send_keys(search_item)
search_button = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search_button.click()   

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [47]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the laptop
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [50]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Ratings = []
No_Ratings = []
Price = []
Return = []
Expected_Delivery = []
Availability = []
Other_Details = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping data for product name
    try:
        prod=driver.find_element_by_xpath("//span[@id='productTitle']")
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping data for brand name
    try:
        brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
     #Scraping data for Ratings
    try:
        rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
        Ratings.append(rat.get_attribute("title"))   
    except NoSuchElementException as e:
        Ratings.append("-")
             
    #Scraping data for No of Ratings
    try:
        no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']/span")
        No_Ratings.append(no_rat.text)
    except NoSuchElementException as e:
        No_Ratings.append("-")
        
        
    #Scraping data for Price
    try:
        pri=driver.find_element_by_xpath("//span[@class='a-price a-text-price a-size-medium apexPriceToPay']")
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping data for Return/Exchange
    try:
        ret=driver.find_element_by_xpath("//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping data for Expected_Delivary
    try:
        delivary=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping data for Availability
    try:
        avai=driver.find_element_by_xpath("//div[@id='availability']/span")
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
              
    #Scraping data for Other_Details
    try:
        details=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        Other_Details.append(details.text)
    except NoSuchElementException as e:
        Other_Details.append("-")
        
        
#DATA FRAMEING
Laptop=pd.DataFrame({})
Laptop['Name'] = Product_name
Laptop['Brand'] = Brand_name
Laptop['Rating'] = Ratings
Laptop['No of Ratings'] = No_Ratings
Laptop['Price'] = Price
Laptop['Return'] = Return
Laptop['Expected_Delivery'] = Expected_Delivery
Laptop['Availability'] = Availability
Laptop['Other_Details'] = Other_Details
Laptop['Urls'] = urls

In [51]:
#Printing the dataframe
Laptop

,Name,Brand,Rating,No of Ratings,Price,Return,Expected_Delivery,Availability,Other_Details,Urls
0,HP Chromebook 14 Intel Celeron N4020-4GB SDRAM...,Visit the HP Store,3.7 out of 5 stars,930 ratings,"₹25,990.00",7 Days Replacement,"Monday, Jan 24",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/gp/slredirect/picassoRed...
1,"HP Pavilion 14, Ryzen 5- 8GB RAM/512GB SSD 14 ...",Visit the HP Store,4.3 out of 5 stars,201 ratings,"₹53,990.00",7 Days Replacement,Jan 22 - 30,Usually dispatched in 3 days.,Processor: AMD Ryzen 5 5500U (up to 4.0 GHz ma...,https://www.amazon.in/gp/slredirect/picassoRed...
2,"Lenovo Chromebook 14e 14.0"" FHD Business Lapto...",Visit the Lenovo Store,3.4 out of 5 stars,20 ratings,"₹19,990.00",7 Days Replacement,"Monday, Jan 24",In stock.,"Light, portable, rugged, productive, MIL-STD-8...",https://www.amazon.in/Lenovo-Chromebook-Busine...
3,HP Chromebook 14 Intel Celeron N4020-4GB SDRAM...,Visit the HP Store,3.7 out of 5 stars,930 ratings,"₹25,990.00",7 Days Replacement,"Monday, Jan 24",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/HP-Chromebook-14a-na0003...
4,Lenovo IdeaPad Slim 1 Intel Celeron N4020 11.6...,Visit the Lenovo Store,4.0 out of 5 stars,228 ratings,-,7 Days Replacement,"Monday, Jan 24",In stock.,Free upgrade to Windows 11 when available* Dis...,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...
...,...,...,...,...,...,...,...,...,...,...
64,Lenovo IdeaPad Slim 5 11th Gen Intel Core i5 1...,Visit the Lenovo Store,4.3 out of 5 stars,114 ratings,-,7 Days Replacement,"Friday, Jan 28",Only 1 left in stock.,Processor: 11th Gen Intel Tiger Lake Core i5-1...,https://www.amazon.in/Lenovo-Keyboard-Fingerpr...
65,Dell 14 (2021) Thin & Light Intel i3-1005G1 La...,Visit the Dell Store,4.3 out of 5 stars,504 ratings,"₹39,990.00",7 Days Replacement,"Wednesday, Jan 26",In stock.,Processor: 10th Generation Intel Core i3-1005G...,https://www.amazon.in/Dell-i3-1005G1-Integrate...
66,Acer Extensa 15 Thin & Light Intel Processor P...,Visit the Acer Store,4.2 out of 5 stars,4 ratings,"₹31,488.00",7 Days Replacement,"Monday, Feb 7",Only 1 left in stock.,𝗥𝗲𝗳𝗶𝗻𝗲𝗱 𝗗𝗲𝘀𝗶𝗴𝗻 𝗮𝗻𝗱 𝗟𝗶𝗴𝗵𝘁 𝗪𝗲𝗶𝗴𝗵𝘁:The latest Ext...,https://www.amazon.in/Acer-Processor-Business-...
67,Acer Aspire 5 A514-54 Thin and Light Laptop | ...,Visit the Acer Store,4.0 out of 5 stars,134 ratings,"₹53,990.00",7 Days Replacement,"Wednesday, Jan 26",In stock.,Powerful Productivity: 11th Generation Intel C...,https://www.amazon.in/gp/slredirect/picassoRed...


# 4: Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by "-". Save your results in a dataframe and CSV.¶

In [57]:
# Connecting to the webdriver
driver = webdriver.Chrome('chromedriver.exe')

In [58]:
# Getting the webpage of mentioned url 
url = 'https://www.flipkart.com/'
driver.get(url)

In [59]:
# Closing Login popup button
login_X_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()

In [60]:
# Searching for web element
search_bar = driver.find_element_by_xpath("//input[@class='_3704LK']")
# Sending keys to search product
search_bar.send_keys("pixel 4A")

In [61]:
# Locating the search button using xpath
search_btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
# Clicking on search button
search_btn.click()

In [49]:
# Fetching 1st page URLs of smartphone
page1_url = []   #empty list
urls = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for url in urls:
    page1_url.append(url.get_attribute('href'))

In [50]:
len(page1_urls)

24

In [51]:
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Display Resolution"] = []
Smartphones["Processor"] = []
Smartphones["Processor Cores"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [52]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
    
    #Scraping display resolution data
    try:
        dires_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if dires_tags.text != "Display Features" : raise NoSuchElementException
        disp_reso = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')    
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
    
    
    
    #Scraping Processor data
    try:
        pro_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_tags.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
        
    
    
    #Scraping Processor core data    
    try:                                                                                     
        core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_tags.text != "Processor Core" :
            core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_tags.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        Smartphones['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
    
    
    
    #Scraping battery capacity data
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=c1ffd6d6-0e3c-4be8-8f8d-bba8063a3dfb.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=oyjcwacow00000001642772202638&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/google-pixel-3a-clearly-white-64-gb/p/itmfgk4jfgstaack?pid=MOBFFGFPJSCEXMSG&lid=LSTMOBFFGFPJSCEXMSGODGRZE&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=c1ffd6d6-0e3c-4be8-8f8d-bba8063a3dfb.MOBFFGFPJSCEXMSG.SEARCH&ppt=hp&ppn=homepage&ssid=oyjcwacow00000001642772202638&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/micromax-x419/p/itm74bbe143d348e?pid=MOBG729CJHY9MFVP&lid=LSTMOBG729CJHY9MFVPCIYAPE&marketplace=FLIPKART&q=pixel&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=SEARCH&iid=c1

In [53]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]), len(Smartphones["Primary Camera"]),len(Smartphones["Secondary Camera"]),len(Smartphones["Display Size"]),len(Smartphones["Display Resolution"]),len(Smartphones["Processor"]),len(Smartphones["Processor Cores"]),len(Smartphones["Battery Capacity"]),len(Smartphones["Price"]),len(Smartphones["URL"]))

24 24 24 24 24 24 24 24 24 24 24 24 24 24


In [54]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,"₹27,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Google,Pixel 3a,Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,"₹39,999",https://www.flipkart.com/google-pixel-3a-clear...
2,Micromax,X419,COCOA BROWN,32 MB,32 MB,-,-,4.5 cm (1.77 inch),128 x 160$$pixel,-,-,-,₹996,https://www.flipkart.com/micromax-x419/p/itm74...
3,GFive,U707,Black Red,32 MB,32 MB,0.3MP Rear Camera,-,4.57 cm (1.8 inch),320 x 480$$pixel,-,-,1000 mAh,₹749,https://www.flipkart.com/gfive-u707/p/itmfy49y...
4,REDMI,10 Prime,Astral White,4 GB,64 GB,50MP + 8MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),2400 x 1080$$pixel,Helio G88,Octa Core,6000 mAh,"₹13,999",https://www.flipkart.com/redmi-10-prime-astral...
5,SAMSUNG,Galaxy M52 5G,Blazing Black,6 GB,128 GB,64MP + 12MP + 5MP,32MP Front Camera,17.02 cm (6.7 inch),1080 x 2400$$pixel,Qualcomm SM7325 Snapdragon,Octa Core,5000 mAh,"₹25,782",https://www.flipkart.com/samsung-galaxy-m52-5g...
6,KARBONN,KX3,"Black,Red",32 MB,32 MB,0.3MP Rear Camera,-,4.57 cm (1.8 inch),128 x 160$$pixel,-,-,800 mAh,₹920,https://www.flipkart.com/karbonn-kx3/p/itm60cf...
7,I,K 19 CAr,Gold,32 MB,32 MB/64 MB,-,-,4.5 cm (1.77 inch),120 x 240$$Pixel,-,-,800 mAh,"₹1,299",https://www.flipkart.com/kall-k-19-car/p/itmd7...
8,GFive,U873,Black Red,32 MB,32 MB,0.3MP Rear Camera,-,4.57 cm (1.8 inch),320 x 480$$pixel,-,-,1000 mAh,₹749,https://www.flipkart.com/gfive-u873/p/itm7f11a...
9,I,K78,Green,32 MB,32 MB,Primary Camera,-,6.1 cm (2.4 inch),240 x 320$$pixel,-,-,2500 mAh,"₹1,092",https://www.flipkart.com/i-kall-k78/p/itm67962...


In [56]:
df.to_csv('SmartPhone_details.csv', index=False, sep=',') #saving the fetched data in csv file
print("File saved")

File saved


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes

In [64]:
#Activating the chrome browser with specified url
driver = webdriver.Chrome("chromedriver.exe")
# getting images.google.com
url = "https://images.google.com/"
#Creating empty list and giving search items as list and creating loop
urls = []    
data = []
search_item = ["fruits", "cars", "Machine Learning","Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    search_bar = driver.find_element_by_tag_name("input") #Xpath for search bar
    
    search_bar.send_keys(str(item))      #sending key word for search item
    
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,10)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [65]:
# Connecting to the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# Getting mentioned url and opening google maps web page
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

# entering the city name in search bar 
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id("searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)

# Sending keys to find cities
search_bar.send_keys(City) 

# Checking for webelement and Clicking on search button
search_btn = driver.find_element_by_id("searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(2)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Bangalore
URL Extracted:  https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.9695744,77.57824,12z/data=!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627
Latitude = 12.9695744, Longitude = 77.57824


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)  from trak.in.

In [77]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

In [78]:
# opening the url track.in
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [79]:
#Getting xpath for funding deals and clicking on the button
fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [96]:
# Empty lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry/Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []

for i in range(54,67):

    # Scraping data of Date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        fund_deals['Date'].append(d.text)
            # Scraping data of Startup Name
    startup_name= driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in startup_name:
        fund_deals['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for ind in industry:
        fund_deals['Industry/Vertical'].append(ind.text)
    
    # Scraping data of Sub-Vertical
    sub_vertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sub_vertical:
        fund_deals['Sub-Vertical'].append(sv.text)

    # Scraping data of Location
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in location:
        fund_deals['Location'].append(loc.text)
    
    # Scraping data of Investor
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for invest in investor:
        fund_deals['Investor'].append(invest.text)
        
    # Scraping data of Investment Type
    investment_type = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invtype in investment_type:
        fund_deals['Investment Type'].append(invtype.text)
        
    # Scraping data of Amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amount:
        fund_deals['Amount(in USD)'].append(amt.text)


In [97]:
# Creating DataFrame for scraped data
fund_data = pd.DataFrame(fund_deals)
fund_data

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [131]:
# Connecting to the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

In [132]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [133]:
#searching for best laptop
best_gam_laptops = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [136]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []

In [139]:
# Scraping the data of laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
#Scraping the data of operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass

#Scraping data of Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


#Scraping data of memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


#Scraping data of weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass

#Scraping data of dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [140]:
#DATA FRAMEING
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [141]:
#Printing data frame
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
1,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
2,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 268,990"
3,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"Rs 342,989"
4,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 319,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
7,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 57,900"


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be  scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [142]:
driver=webdriver.Chrome("chromedriver")
url='https://www.forbes.com/real-time-billionaires/#623cbcd03d78' 
driver.get(url)

In [143]:
billionaries_details=[]
for i in driver.find_elements_by_xpath("//tr[@class='base ng-scope']"):
        billionaries_details.append(i.text.replace("\n"," "))
billionaries_details

['1 Elon Musk $244.2 B ▶ $11.8 B | -4.62% 50 Tesla, SpaceX United States',
 '2 Bernard Arnault & family $190.7 B ▶ $1.8 B | -0.94% 72 LVMH France',
 '3 Jeff Bezos $168.5 B ▶ $9 B | -5.08% 58 Amazon United States',
 '4 Bill Gates $131.3 B ▶ $1.4 B | -1.03% 66 Microsoft United States',
 '5 Larry Ellison $112.6 B ▶ $1.3 B | -1.18% 77 software United States',
 '6 Larry Page $111.9 B ▶ $2.5 B | -2.21% 48 Google United States',
 '7 Warren Buffett $110.5 B ▶ $2.1 B | -1.85% 91 Berkshire Hathaway United States',
 '8 Sergey Brin $107.9 B ▶ $2.4 B | -2.19% 48 Google United States',
 '9 Mark Zuckerberg $107.5 B ▶ $4.7 B | -4.15% 37 Facebook United States',
 '10 Mukesh Ambani $95.2 B ▶ $27 M | -0.03% 64 diversified India',
 '11 Steve Ballmer $94.7 B ▶ $1.5 B | -1.54% 65 Microsoft United States',
 '12 Gautam Adani & family $92.1 B ▶ $401 M | -0.43% 59 infrastructure, commodities India',
 "13 Francoise Bettencourt Meyers & family $84.2 B ▶ $1.1 B | -1.31% 68 L'Oréal France",
 '14 Carlos Slim Helu & 

# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted  from any YouTube Video.

In [152]:
driver=webdriver.Chrome("chromedriver")

In [153]:
youtube_pages = "https://www.youtube.com/"
driver.get(youtube_pages)

In [157]:
search = driver.find_element_by_xpath('//input') 
search.send_keys("Unbox Karnataka")
driver.find_element_by_xpath('//h2[@class="style-scope ytd-comments-header-renderer"]').click()

In [158]:
comments=[]
for i in range(10):
    for j in driver.find_elements_by_id("content-text"):
        comments.append(j.text)
comments

['OFFERS FOR UNBOX KARNATAKA SUBSCRIBERS:\n1. Complimentary Jeep-off-road for a group of 8 and more.\n2. 20% Discount on Weekdays on Camping & Cottage Stay. (Valid from Monday to Thursday)\n3. Weekend Lockdown Offer – Stay for 3 Nights & pay for 2 Nights only (valid during weekend      lockdown only)',
 '@Unbox Karnataka hats off to your team in the way you research and showcase such places really enjoyed the scenic beauty of nature along with local food... please showcase some more hospitality videos',
 "Exceptional bro, will definitely share this with my pals abt this n will visit in comit days\n\nHats of to ur team for exploring these places, which people like us don't know where to go during break",
 'Peaceful vlog ....if u want Peace must watch this video \n1) Panorama shots is amazing .. harshvardhan done good job\n2)  good Hospitality \n3) good food  veg & nonveg ',
 'Beautiful place 😍👍',
 'We are already visited there.there hospitality is really exllent.',
 'Nice presentation. 

In [159]:
upvotes=[]
for i in range(10):
    try:
        for j in driver.find_elements_by_xpath("//span[@id='vote-count-middle']"):
            upvotes.append(j.text)
    except:
        upvotes.append("-")
upvotes

['14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '14',
 '7',
 '',
 '2',
 '',
 '',
 '',
 '',
 '2',
 '',
 '',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 

In [160]:
time=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text above-comment style-scope ytd-comment-renderer']"):
        time.append(j.text)
time

['10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '9 days ago',
 '8 days ago',
 '9 days ago (edited)',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '9 days ago',
 '8 days ago',
 '9 days ago (edited)',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '9 days ago',
 '8 days ago',
 '9 days ago (edited)',
 '10 days ago',
 '8 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '9 days ago',
 '10 days ago',
 '10 days ago',
 '10 days ago',
 '9 d

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [184]:
driver=webdriver.Chrome("chromedriver")
url='https://www.hostelworld.com/'             #passing url
driver.get(url)              
search = driver.find_element_by_xpath("//input")
search.send_keys("London")
submit=driver.find_element_by_xpath("//button[@id='search-button']")
submit.click()

In [185]:
hostel_name=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='title-row']//h2"): #fetching hostel_name tag using xpath
        hostel_name.append(j.text)
hostel_name

["St Christopher's Village",
 'Mornington Camden',
 'Smart Russell Square Hostel',
 'Smart Camden Inn Hostel',
 'Selina Camden',
 'Queen Elizabeth Chelsea',
 'Pickwick Hall',
 'New Cross Inn Hostel',
 'No.8 Willesden Hostel London',
 'Safestay London Elephant & Castle',
 'The London Home Hostel',
 'No.8 Seven Sisters',
 'Park Villa',
 'Astor Hyde Park',
 "Wombat's The City Hostel London",
 "St Christopher's Greenwich",
 "St Christopher's Oasis-Female Only",
 "St Christopher's Camden",
 "St Christopher's Shepherds Bush",
 "St Christopher's Hammersmith",
 "St Christopher's Inn - Liverpool Street",
 "St Christopher's Inn - London Bridge",
 'PubLove @ The White Ferry, Victoria',
 'Smart Hyde Park Inn Hostel',
 'Smart Hyde Park View Hostel',
 'Safestay London Kensington Holland Park',
 'Urbany Hostel London',
 'Astor Kensington',
 'Phoenix Hostel',
 'Prime Backpackers Angel',
 'YHA London Earls Court',
 "St Christopher's Village",
 'Mornington Camden',
 'Smart Russell Square Hostel',
 'Smar

In [186]:
distance=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//span[@class='description']"):
        distance.append(j.text)
distance

['Hostel - 4.1km from city centre',
 'Hostel - 2.6km from city centre',
 'Hostel - 4.4km from city centre',
 'Hostel - 5.5km from city centre',
 'Hostel - 5.7km from city centre',
 'Hostel - 2.3km from city centre',
 'Hostel - 6.1km from city centre',
 'Hostel - 10km from city centre',
 'Hostel - 1.7km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 9km from city centre',
 'Hostel - 6.3km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 7.6km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 7km from city centre',
 'Hostel - 7.5km from city centre',
 'Hostel - 3.2km from city centre',
 'Hostel - 1.8km from city centre',
 'Hostel - 2.4km from city centre',
 'Hostel - 5km from city centre',
 'Hostel - 5km from city centre',
 'Hostel - 5.9km from city centre',
 'Hostel - 5.4km from city centre',
 'Hostel - 4.9km from city centre',
 'Hostel - 4.2km from city centre',
 

In [187]:
ratings=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='score orange big']"):
        ratings.append(j.text)
ratings

['8.3',
 '6.7',
 '8.5',
 '9.1',
 '7.5',
 '9.0',
 '7.4',
 '7.6',
 '8.0',
 '6.7',
 '8.3',
 '8.5',
 '8.8',
 '7.1',
 '7.9',
 '7.2',
 '8.3',
 '7.8',
 '8.4',
 '8.0',
 '8.4',
 '6.8',
 '7.6',
 '8.2',
 '9.6',
 '7.3',
 '7.8',
 '9.2',
 '9.7',
 '8.3',
 '6.7',
 '8.5',
 '9.1',
 '7.5',
 '9.0',
 '7.4',
 '7.6',
 '8.0',
 '6.7',
 '8.3',
 '8.5',
 '8.8',
 '7.1',
 '7.9',
 '7.2',
 '8.3',
 '7.8',
 '8.4',
 '8.0',
 '8.4',
 '6.8',
 '7.6',
 '8.2',
 '9.6',
 '7.3',
 '7.8',
 '9.2',
 '9.7',
 '8.3',
 '6.7',
 '8.5',
 '9.1',
 '7.5',
 '9.0',
 '7.4',
 '7.6',
 '8.0',
 '6.7',
 '8.3',
 '8.5',
 '8.8',
 '7.1',
 '7.9',
 '7.2',
 '8.3',
 '7.8',
 '8.4',
 '8.0',
 '8.4',
 '6.8',
 '7.6',
 '8.2',
 '9.6',
 '7.3',
 '7.8',
 '9.2',
 '9.7',
 '8.3',
 '6.7',
 '8.5',
 '9.1',
 '7.5',
 '9.0',
 '7.4',
 '7.6',
 '8.0',
 '6.7',
 '8.3',
 '8.5',
 '8.8',
 '7.1',
 '7.9',
 '7.2',
 '8.3',
 '7.8',
 '8.4',
 '8.0',
 '8.4',
 '6.8',
 '7.6',
 '8.2',
 '9.6',
 '7.3',
 '7.8',
 '9.2',
 '9.7',
 '8.3',
 '6.7',
 '8.5',
 '9.1',
 '7.5',
 '9.0',
 '7.4',
 '7.6',
 '8.0',


In [188]:
total_reviews=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='reviews']"):
        total_reviews.append(j.text) 
total_reviews

['44 Total Reviews',
 '9567 Total Reviews',
 '2738 Total Reviews',
 '14 Total Reviews',
 '3212 Total Reviews',
 '2275 Total Reviews',
 '1055 Total Reviews',
 '4645 Total Reviews',
 '4025 Total Reviews',
 '253 Total Reviews',
 '3684 Total Reviews',
 '854 Total Reviews',
 '11385 Total Reviews',
 '13369 Total Reviews',
 '3155 Total Reviews',
 '620 Total Reviews',
 '3731 Total Reviews',
 '641 Total Reviews',
 '3970 Total Reviews',
 '373 Total Reviews',
 '3254 Total Reviews',
 '187 Total Reviews',
 '6261 Total Reviews',
 '4517 Total Reviews',
 '1107 Total Reviews',
 '187 Total Reviews',
 '6341 Total Reviews',
 '3480 Total Reviews',
 '531 Total Reviews',
 '1383 Total Reviews',
 '44 Total Reviews',
 '9567 Total Reviews',
 '2738 Total Reviews',
 '14 Total Reviews',
 '3212 Total Reviews',
 '2275 Total Reviews',
 '1055 Total Reviews',
 '4645 Total Reviews',
 '4025 Total Reviews',
 '253 Total Reviews',
 '3684 Total Reviews',
 '854 Total Reviews',
 '11385 Total Reviews',
 '13369 Total Reviews',
 '

In [189]:
privatesfromprice=[]
Dormsfromprice=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='price title-5']"):
         for k in driver.find_elements_by_xpath("//div[@class='price title-5']"):
                privatesfromprice.append(j.text) 
                Dormsfromprice.append(k.text)
print(privatesfromprice)
print(Dormsfromprice)

['Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs3272', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552', 'Rs4552',

In [190]:
urls=[]
for i in driver.find_elements_by_xpath("//h2[@class='title title-6']//a"):
    urls.append(i.get_attribute("href"))
urls

['https://www.hostelworld.com/pwa/hosteldetails.php/Mornington-Camden/London/285882?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Smart-Russell-Square-Hostel/London/29412?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Smart-Camden-Inn-Hostel/London/596?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Selina-Camden/London/310274?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Queen-Elizabeth-Chelsea/London/50443?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Pickwick-Hall/London/7502?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/New-Cross-Inn-Hostel/London/15204?from=2022-01-23&to=2022-01-26&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/No-8-Willesden-Hostel-London/London/18420?from=2022-01-23&to=202

In [191]:
overall_review=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='keyword']//span"):
        overall_review.append(j.text)
overall_review

['Fabulous',
 'Fabulous',
 'Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Superb',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Rating',
 'Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Very Good',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Good',
 'Very Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Very Good',
 'Superb',
 'Superb',
 'Fabulous',
 'Fabulous',
 'Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Superb',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Rating',
 'Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Very Good',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Good',
 'Very Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Very Good',
 'Superb',
 'Superb',
 'Fabulous',
 'Fabulous',
 'Good',
 'Fabulous',
 'Superb',
 'Very Good',
 'Superb',
 'Very Good',
 'Very Good',
 'Fabulous',
 'Rating',
 'Good',
 'Fabulous',
 'Fabulous',
 'Fabulous',
 'Very Good',
 'Very Good',
 'Very Goo

In [208]:
facilities=[]

for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='facilities-container']"):
        facilities.append(j.text) 
facilities

['Facilities\nFree\nLinen Included Free City Maps Free WiFi Free Internet Access Free City Tour\nGeneral\nSecurity Lockers Key Card Access Common Room Elevator Breakfast Not Included Air Conditioning Adaptors Book Exchange Hot Showers Dryer Reading Light Hair Dryers For Hire Hair Dryers Iron / Ironing Board Towels Not Included Flexible NRR Follows Covid-19 sanitation guidance\nServices\nInternet access Laundry Facilities Towels for hire Luggage Storage 24 Hour Reception Tours/Travel Desk 24 Hour Security Housekeeping\nFood & Drink\nRestaurant Bar Meals Available Vending Machines Tea & Coffee Making Facilities\nEntertainment\nNightclub Games Room Wi-Fi',
 'Facilities\nFree\nLinen Included Free City Maps Free WiFi Free Internet Access Free City Tour\nGeneral\nSecurity Lockers Key Card Access Common Room Elevator Breakfast Not Included Air Conditioning Adaptors Book Exchange Hot Showers Dryer Reading Light Hair Dryers For Hire Hair Dryers Iron / Ironing Board Towels Not Included Flexible 

In [213]:
job_desc=[]        

In [217]:
for i in range(10):
    for j in driver.find_elements_by_xpath("//h4[@class='title-4']"):
        job_desc.append(j.text)

In [218]:
job_desc 

['Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availability',
 'Reviews & Ratings',
 'Facilities',
 'Awards',
 'Hostelworld says',
 'Property Description',
 'Check Availabili